#### 1- DES

In [2]:
# Step 1: Input a 16-character hexadecimal key
hex_key = "133457799BBCDFF1"

# Validate input
if len(hex_key) != 16 or any(c not in "0123456789abcdefABCDEF" for c in hex_key):
    print(" Invalid input! Please enter only 0-9 or A-F (16 characters).")
    exit()

print(" Hex Key Entered:", hex_key)

# Step 2: Convert the hex key to binary (64-bit)
bin_key = bin(int(hex_key, 16))[2:].zfill(64)

print(" Binary Key (64-bit):", bin_key)

# Step 3: Apply PC-1 to drop parity bits
#PC-1 removes parity bits and rearranges the remaining bits to form a 56-bit key.
PC1 = [
    57, 49, 41, 33, 25, 17, 9,
    1, 58, 50, 42, 34, 26, 18,
    10, 2, 59, 51, 43, 35, 27,
    19, 11, 3, 60, 52, 44, 36,
    63, 55, 47, 39, 31, 23, 15,
    7, 62, 54, 46, 38, 30, 22,
    14, 6, 61, 53, 45, 37, 29,
    21, 13, 5, 28, 20, 12, 4
]
for i in PC1:
    bin_key[i-1]
    perm_key= "".join(bin_key)
print("56-bit Permuted Key:", perm_key)
'''
perm_key = "".join([bin_key[i - 1] for i in PC1])
print("56-bit Permuted Key:", perm_key)
'''
# Step 4: Split the key into two halves (each 28 bits)
L = perm_key[:28]
R = perm_key[28:]
print("L0 =", L)
print("R0 =", R)

# Number of left shifts per round
ROTATIONS = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

def left_shift(bits, n):
    return bits[n:] + bits[:n]

    
# PC-2 Table (Permuted Choice 2)
PC2 = [
    14, 17, 11, 24, 1, 5,
    3, 28, 15, 6, 21, 10,
    23, 19, 12, 4, 26, 8,
    16, 7, 27, 20, 13, 2,
    41, 52, 31, 37, 47, 55,
    30, 40, 51, 45, 33, 48,
    44, 49, 39, 56, 34, 53,
    46, 42, 50, 36, 29, 32
]

# Generate 16 subkeys
subkeys = []

for i in range(16):
    L = left_shift(L, ROTATIONS[i])
    R = left_shift(R, ROTATIONS[i])
    combined = L + R
    subkey = "".join([combined[j - 1] for j in PC2])
    subkeys.append(subkey)
    print(f"Round {i+1} key: {subkey}")


 Hex Key Entered: 133457799BBCDFF1
 Binary Key (64-bit): 0001001100110100010101110111100110011011101111001101111111110001
56-bit Permuted Key: 0001001100110100010101110111100110011011101111001101111111110001
L0 = 0001001100110100010101110111
R0 = 100110011011101111001101111111110001
Round 1 key: 011000100111110010011010011001011101111101010101
Round 2 key: 000101000111001010100101111111101001101111011000
Round 3 key: 100100110001110101110100110101011111011101111111
Round 4 key: 110011000110001011100101011110111011111011101000
Round 5 key: 100100111101011100101100111110001111110100111111
Round 6 key: 110010000001001111100011001011110111111010111110
Round 7 key: 101100011101101001101101111110010111100111110011
Round 8 key: 100000010111001111000010101001111100101000111111
Round 9 key: 011101100010001001011110101100110100111011011010
Round 10 key: 001011101101110000010000110111011011001100010111
Round 11 key: 010011100010100101111110101100110110011111101100
Round 12 key: 101011101110010000

#### 2- RSA

In [ ]:
import math
import random

def is_prime(num):
    if num< 2:
        return False
    for i in range(2, int(math.sqrt(num))+1):
        if num %i == 0:
            return False
    return True


def generate_prime(bits):
    while True:
        num = random.getrandbits(bits)
        if is_prime(num):
            return num

bit_length=10
p= generate_prime(bit_length)
q=generate_prime(bit_length)

def gcd(a,b):
    while b:
        a,b=b,a%b
    return a

n= p*q
euler = (p-1)*(q-1)
e = random.randrange(2,euler)
while gcd(e,euler)!=1:
    e=random.randrange(2,euler)
print(f"public key = ({n},{e})")


for i in range(1, euler):
    if (i * e) % euler == 1:
        d = i
        break

print(f"Public key: ( {e} , {n})")
print(f"Private key: ( {d} , {n})")

#  Convert message to numbers
message = "hi"
message_ascii = [ord(ch) for ch in message]
print("\nMessage ASCII:", message_ascii)

#Encrypt each character
cipher = [(m ** e) % n for m in message_ascii]
print("Encrypted Message:", cipher)

# Decrypt each character
decrypted = [(c ** d) % n for c in cipher]
decrypted_text = ''.join(chr(num) for num in decrypted)

print("Decrypted Message:", decrypted_text)


public key = (180197,119017)
Public key: ( 119017 , 180197)
Private key: ( 111193 , 180197)

Message ASCII: [104, 105]
Encrypted Message: [105813, 66132]
Decrypted Message: hi


#### 3- Diffe-Hellman

In [ ]:
from sympy import randprime, primitive_root
import random

# Step 1: Publicly shared p and g (generated dynamically)
p = randprime(1000, 5000)
g = primitive_root(p)
print("Publicly shared values:\np =", p, "\ng =", g)

# Step 2: Each side picks a private key
a = random.randint(2, p-2)  # Alice's private key
b = random.randint(2, p-2)  # Bob's private key

# Step 3: Compute public keys
A = pow(g, a, p)  # Alice's public key
B = pow(g, b, p)  # Bob's public key

print("\nAlice's Public Key (A):", A)
print("Bob's Public Key (B):", B)

# Step 4: Exchange and compute shared secret
shared_secret_A = pow(B, a, p)
shared_secret_B = pow(A, b, p)

print("\nShared Secret (computed by Alice):", shared_secret_A)
print("Shared Secret (computed by Bob):", shared_secret_B)


Publicly shared values:
p = 4283 
g = 2

Alice's Public Key (A): 432
Bob's Public Key (B): 3014

Shared Secret (computed by Alice): 939
Shared Secret (computed by Bob): 939


#### 4- Message Digest 5 (MD5)

In [ ]:
import numpy as np


def padding_md5(msg):
    # تحويل الرسالة إلى بايتات
    msg_bytes = bytearray(msg.encode('utf-8'))
    msg_len_bits = len(msg_bytes) * 8 # الطول بالبتات
    blocks=np.floor(msg_len_bits/512)
    len_block=msg_len_bits-(blocks*512)
    if len_block<448:
        pad=512-(len_block+64)
    else:
        #  بلوك جديد
        pad=(512-(len_block+64))+512
    return pad

s1=[7,12,17,22]
t=[
    0xd76aa478, 0xe8c7b756, 0x242070db, 0xc1bdceee,
    0xf57c0faf, 0x4787c62a, 0xa8304613, 0xfd469501,
    0x698098d8, 0x8b44f7af, 0xffff5bb1, 0x895cd7be,
    0x6b901122, 0xfd987193, 0xa679438e, 0x49b40821
]
def first_round(a, b, c, d, k, i):
    f = (b & c) | (~b & d)
    f = (f + a + t[i] + k) & 0xFFFFFFFF
    f = (f << s1[i % 4]) | (f >> (32 - s1[i % 4]))
    f = (f + b) & 0xFFFFFFFF
    return f

#### 6- Secure Hash Algorithm 1 (SHA-1)

In [39]:
import struct
 

# Helper Function:
def left_rotate(n, b):
    return ((n << b) | (n >> (32 - b))) & 0xFFFFFFFF

message = "Hello"
data = message.encode('utf-8') 

# Calculating the bit-length of the message:
ml = len(data) * 8

# Appending 1 bit at the end of the converted message:
data += b'\x80'

### 1. Padding:
while (len(data) * 8) % 512 != 448:
    data += b'\x00'

### 2. Message Length:
data += struct.pack('>Q', ml)

### 3. Initialize Variables:
h0 = 0x67452301
h1 = 0xEFCDAB89
h2 = 0x98BADCFE
h3 = 0x10325476
h4 = 0xC3D2E1F0

### 4. Process the Message:
for i in range(0, len(data), 64):
    chunk = data[i:i + 64]

    words = [0] * 80
    for j in range(0, 64, 4):
        words[j // 4] = struct.unpack('>I', chunk[j:j + 4])[0]

    ### 5. Extend the Message Schedule
    for j in range(16, 80):
        words[j] = left_rotate((words[j - 3] ^ words[j - 8] ^ words[j - 14] ^ words[j - 16]), 1)

    ### 6. Initialize Temporary Variables:
    a, b, c, d, e = h0, h1, h2, h3, h4

    ### 7. Compression Function:
    for j in range(80):
        if 0 <= j <= 19:
            f = (b & c) | ((~b) & d)
            k = 0x5A827999
        elif 20 <= j <= 39:
            f = b ^ c ^ d
            k = 0x6ED9EBA1
        elif 40 <= j <= 59:
            f = (b & c) | (b & d) | (c & d)
            k = 0x8F1BBCDC
        else:
            f = b ^ c ^ d
            k = 0xCA62C1D6

        temp = left_rotate(a, 5) + f + e + k + words[j] & 0xFFFFFFFF
        e, d, c, b, a = d, c, left_rotate(b, 30), a, temp

    ### 8. Update Hash Values:
    h0 = (h0 + a) & 0xFFFFFFFF
    h1 = (h1 + b) & 0xFFFFFFFF
    h2 = (h2 + c) & 0xFFFFFFFF
    h3 = (h3 + d) & 0xFFFFFFFF
    h4 = (h4 + e) & 0xFFFFFFFF
        
### 9. Final Hash Value:
print('%08x%08x%08x%08x%08x' % (h0, h1, h2, h3, h4))


f7ff9e8b7bb2e09b70935a5d785e0cc5d9d0abf0


#### 6- Data Signature Standard (DSS)

In [9]:
from ecdsa import SigningKey

message = "Message".encode()
private_key = SigningKey.generate()
public_key = private_key.get_verifying_key()

signature = private_key.sign(message)
print(signature)

verification = public_key.verify(signature, message)

if verification:
    print("Verified.")
else:
    print("Not verified.")


b'\x15G4Y\xbac\xf0\xa2\xe8oRs\xd1\x1f\r\x81i\t\xd8e&\xdd:\xedg?|Bh\xb2>#\xe8\xa5\xbc=\x90\xc7X3\xe6?z\xd1\xf4Nz\x16'
Verified.


# SDES

In [4]:
# Function to rearrange bits according to a pattern
def permute(bits, pattern):
    return ''.join(bits[i - 1] for i in pattern)

# Function for circular left shift
def left_shift(bits, n):
    return bits[n:] + bits[:n]


def key_generation(key):
    # Step 1: P10
    P10 = [3,5,2,7,4,10,1,9,8,6]
    P8  = [6,3,7,4,8,5,10,9]
    
    key = permute(key, P10)
    left, right = key[:5], key[5:]
    
    # Left shift by 1
    left, right = left_shift(left, 1), left_shift(right, 1)
    K1 = permute(left + right, P8)
    
    # Left shift by 2
    left, right = left_shift(left, 2), left_shift(right, 2)
    K2 = permute(left + right, P8)
    
    return K1, K2


key = "1010000010"
K1, K2 = key_generation(key)
print("K1:", K1, "K2:", K2)

def fk(bits, key):
    EP = [4,1,2,3,2,3,4,1]
    S0 = [
        [1,0,3,2],
        [3,2,1,0],
        [0,2,1,3],
        [3,1,3,2]
    ]
    S1 = [
        [0,1,2,3],
        [2,0,1,3],
        [3,0,1,0],
        [2,1,0,3]
    ]
    P4 = [2,4,3,1]

    left, right = bits[:4], bits[4:]
    
    # Step 1: Expand and permute
    right_expanded = permute(right, EP)
    
    # Step 2: XOR with key
    xor_result = ''.join(str(int(a) ^ int(b)) for a,b in zip(right_expanded, key))
    
    # Step 3: Split into 2 halves for S-boxes
    left_xor, right_xor = xor_result[:4], xor_result[4:]
    
    # Step 4: S-box substitution
    def sbox_lookup(bits, sbox):
        row = int(bits[0] + bits[3], 2)
        col = int(bits[1] + bits[2], 2)
        return format(sbox[row][col], '02b')
    
    sbox_output = sbox_lookup(left_xor, S0) + sbox_lookup(right_xor, S1)
    
    # Step 5: XOR with left half and right half
    left_result = ''.join(str(int(a) ^ int(b)^int(c)) for a,b,c in zip(left, sbox_output,right))
    
    # Step 6: Combine halves
    return left_result + right


def encrypt(plaintext, key):
    IP = [2,6,3,1,4,8,5,7]
    IP_inv = [4,1,3,5,7,2,8,6]
    
    # Generate keys
    K1, K2 = key_generation(key)
    
    # Step 1: Initial Permutation
    bits = permute(plaintext, IP)
    #K1="10100010"
    # Step 2: Round 1
    bits = fk(bits, K1)
    
    # Step 3: Swap halves
    bits = bits[4:] + bits[:4]
    
    # Step 4: Round 2
    bits = fk(bits, K2)
    
    # Step 5: Inverse IP
    ciphertext = permute(bits, IP_inv)
    return ciphertext

plaintext = "10100010"
key = "1010000010"
cipher = encrypt(plaintext, key)
print("Ciphertext:", cipher)



K1: 10100100 K2: 01000011
Ciphertext: 00100110
